# Model Building for Food Recommendation

We will implement the TF-IDF algorithm

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Import Gradio for creating web interfaces
import gradio as gr

# Import TF-IDF Vectorizer and cosine similarity from sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# Read the CSV file containing all dishes into a pandas DataFrame
halal = False

if halal:
    all_dishes = pd.read_csv("all_dishes_halal.csv")
else:
    all_dishes = pd.read_csv("all_dishes.csv")



## TF-IDF 

Term Frequency-Inverse Document Frequency (TF-IDF) for each ingredient. Give higher weights to ingredients that are important for a specific dish but less frequent in others, which might be more useful for recommendation purposes.

In [3]:
# Initialize the TF-IDF Vectorizer
tfid = TfidfVectorizer()

# Fit and transform the 'describe' column in the all_dishes DataFrame
tfid_matrix = tfid.fit_transform(all_dishes['describe'])

In [ ]:
# Compute the cosine similarity matrix based on the TF-IDF matrix
cosine_sim = cosine_similarity(tfid_matrix, tfid_matrix)

# Display the cosine similarity matrix
cosine_sim

In [ ]:
# Print the shape of the all_dishes DataFrame
print("Shape of our dishes: ", all_dishes.shape)

# Print the shape of the cosine similarity matrix
print("Shape of cosine similarity matrix: ", cosine_sim.shape)

#(723, 723) meaning we have computed a similarity score for each pair of dishes in our dataset.

## Functions to recommend dishes based on cosine similarity

In [6]:
def rank(title, cosine_sim=cosine_sim, dish_matrix=all_dishes):
    """
    Rank the top 5 similar dishes based on cosine similarity.
    
    Parameters:
    title (str): The name of the dish to find similar dishes for.
    cosine_sim (numpy.ndarray): A 2D array where each element represents the cosine similarity between dishes.
    dish_matrix (pandas.DataFrame): A DataFrame containing dish information, including a 'name' column.
    """
    title = title.lower()  # Convert title to lowercase for case-insensitive matching
    idx = dish_matrix.index[dish_matrix['name'] == title].tolist()  # Get the index of the dish
    match_food = np.array(cosine_sim[idx[0]])  # Get the cosine similarity scores for the dish
    indices = np.argsort(match_food)[-6:][::-1][1:]  # Get the indices of the top 5 similar dishes in descending order
    
    return indices

def print_similar_dishes(title, indices=None, dish_matrix=all_dishes):
    indices = rank(title, cosine_sim=cosine_sim, dish_matrix=all_dishes)  # Get the indices of similar dishes
    print("Top 5 similar dishes to", title, "are:")
    for i in range(len(indices)):
        print(i + 1, ". ", dish_matrix['name'].iloc[indices[i]])  # Print the names of the similar dishes

def rank_for_gradio(title):
    indices = rank(title)  # Get the indices of similar dishes
    similar_dishes = [all_dishes.iloc[i]['name'] for i in indices]  # Get the names of the similar dishes
    formatted_dishes = "\n".join([f"{i + 1}. {dish}" for i, dish in enumerate(similar_dishes)])  # Format the output
    return formatted_dishes


### Testing the results

In [ ]:
print_similar_dishes('Shawarma', all_dishes)

In [ ]:
print_similar_dishes('crispy herb chicken', all_dishes)

In [ ]:
print_similar_dishes('Pho Chay Soup', all_dishes)

In [ ]:
print_similar_dishes('Kale Caesar Salad', all_dishes)

In [ ]:
print_similar_dishes('Adhirasam', all_dishes)

## Create a Gradio interface

In [27]:
# Setup Gradio interface
#css_code = ".gradio-container {background: url(https://cdn.pixabay.com/photo/2024/06/22/08/07/ai-generated-8845695_1280.jpg)}"
# css_code = ".gradio-container {background: url(https://img.freepik.com/free-photo/elevated-view-ingredients-dryfruits-vegetables-black-background_23-2148026898.jpg?w=1480&t=st=1727902059~exp=1727902659~hmac=4f7f4e8f697f78d214ab8f9ee921baa16a01f17683c9b1f44779effcdb546ef9)}"

# iface = gr.Interface(
#     fn=rank_for_gradio,
#     inputs=gr.Dropdown(list(all_dishes["name"]), label="Choose a Dish"),
#     outputs=gr.Text(label="Top 5 Similar Dishes that you might like"),
#     title="Personalized Dish Recommender",
#     description="Select a dish to find the top 5 similar dishes based on their descriptions.",
#     css=css_code,
# )
css_code = """
.gradio-container {
    background: url(https://cdn.pixabay.com/photo/2024/06/22/08/07/ai-generated-8845695_1280.jpg);
}

/* Styling for the title and description box */
.title-desc-box {
    background-color: #f8f9fa;  /* Light grey background */
    border: 1px solid #dee2e6;  /* Grey border */
    border-radius: 10px;        /* Rounded corners */
    padding: 20px;              /* Padding around text */
    margin: 20px;               /* Margin around the box */
}

/* Custom title styling */
h1, h2 {
    color: #333;                /* Dark grey text color */
    text-align: center;         /* Center-align text */
}

/* Custom description styling */
p {
    color: #555;                /* Medium grey text color */
    font-size: 16px;            /* Larger font size */
}
"""

# Interface initialization with modified css
iface = gr.Interface(
    fn=rank_for_gradio,
    inputs=gr.Dropdown(list(all_dishes["name"]), label="Choose a Dish"),
    outputs=gr.Text(label="Top 5 Similar Dishes that you might like"),
    title="<div class='title-desc-box'><h1>Personalized Dish Recommender</h1><p>Select a dish to find the top 5 similar dishes based on their descriptions.</p></div>",
    description="",
    css=css_code,
)


In [ ]:
# Launch the Gradio interface with sharing enabled
iface.launch(share=True)